![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_BartForZeroShotClassification.ipynb)

# Import OpenVINO BartForZeroShotClassification   models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting BartForZeroShotClassification   models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for BartForZeroShotClassification   from BartForZeroShotClassification   and they have to be in `Zero Shot Classification` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers==4.48.3 optimum[openvino]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.6/342.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- To load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- By setting `export=True`, the source model is converted to OpenVINO IR format on the fly.
- We'll use [facebook/bart-large-mnli](https://huggingface.co/facebook/bart-large-mnli) model from HuggingFace as an example and load it as a `OVModelForSequenceClassification`, representing an OpenVINO model.
- In addition to the BartForZeroShotClassification   model, we also need to save the `AutoTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [ ]:
from transformers import AutoTokenizer
from optimum.intel import OVModelForSequenceClassification

MODEL_NAME = "facebook/bart-large-mnli"
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

ort_model = OVModelForSequenceClassification.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'FetchError: Could not fetch resource at https://colab.research.google.com/userdata/get?authuser=0&notebookid=1UNfbUf3s29teZUBaIey-OkzZm0A3xuKO&key=HF_TOKEN: 401  '.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
/usr/local/lib/python3.11/dist-packages/transformers/models/bart/modeling_bart.py:505: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_output.size() != (bsz, self.num_heads, tgt_len, self.head_dim):
/usr/local/lib/python3.11/dist-packages/transformers/modeling_attn_mask_utils.py:88: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if input_shape[-1] > 1 or self.sliding_window is not None:
/usr/local/lib/python3.11/dist-packages/transformers/model

('onnx_models/facebook/bart-large-mnli/tokenizer_config.json',
 'onnx_models/facebook/bart-large-mnli/special_tokens_map.json',
 'onnx_models/facebook/bart-large-mnli/vocab.json',
 'onnx_models/facebook/bart-large-mnli/merges.txt',
 'onnx_models/facebook/bart-large-mnli/added_tokens.json',
 'onnx_models/facebook/bart-large-mnli/tokenizer.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 1596880
-rw-r--r-- 1 root root       1197 Jun 18 01:44 config.json
-rw-r--r-- 1 root root     456318 Jun 18 01:44 merges.txt
-rw-r--r-- 1 root root 1629376728 Jun 18 01:44 openvino_model.bin
-rw-r--r-- 1 root root     988634 Jun 18 01:44 openvino_model.xml
-rw-r--r-- 1 root root        279 Jun 18 01:44 special_tokens_map.json
-rw-r--r-- 1 root root       1243 Jun 18 01:44 tokenizer_config.json
-rw-r--r-- 1 root root    3558642 Jun 18 01:44 tokenizer.json
-rw-r--r-- 1 root root     798293 Jun 18 01:44 vocab.json


- We need to move `vocab.txt` and `merges.txt` from the tokenizer into an `assets` folder, which is where Spark NLP expects to find them. However, before doing that, we first need to convert Hugging Face’s `vocab.json` into a plain `vocab.txt` format, as Spark NLP does not support the JSON format.
- Additionally, we need to extract the `labels` and their corresponding `ids` from the model's config. This mapping will be saved as `labels.txt` inside the same `assets` folder.

In [4]:
!mkdir -p {EXPORT_PATH}/assets

with open(f"{EXPORT_PATH}/assets/labels.txt", "w") as f:
    f.write("\n".join([v for k, v in sorted(ort_model.config.id2label.items())]))

import json
with open(f"{EXPORT_PATH}/vocab.json") as fin, open(f"{EXPORT_PATH}/assets/vocab.txt", "w") as fout:
    fout.writelines(f"{k}\n" for k in json.load(fin))

!mv {EXPORT_PATH}/merges.txt {EXPORT_PATH}/assets/

In [5]:
!ls -l {EXPORT_PATH}/assets

total 852
-rw-r--r-- 1 root root     32 Jun 18 01:44 labels.txt
-rw-r--r-- 1 root root 456318 Jun 18 01:44 merges.txt
-rw-r--r-- 1 root root 407065 Jun 18 01:44 vocab.txt


- Voila! We have our `vocab.txt` and `merges.txt` inside assets directory, along with the extracted labels saved in `labels.txt`.

## Import and Save BartForZeroShotClassification in Spark NLP


- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [6]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 37.1 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `BartForZeroShotClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `BartForZeroShotClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [8]:
from sparknlp.annotator import BartForZeroShotClassification

sequenceClassifier = BartForZeroShotClassification\
  .loadSavedModel(EXPORT_PATH, spark)\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")\
  .setCaseSensitive(False)\
  .setMaxSentenceLength(128)\
  .setCandidateLabels(["urgent", "mobile", "travel", "movie", "music", "sport", "weather", "technology"])

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
sequenceClassifier.write().overwrite().save("./{}_spark_nlp_openvino".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your BartForZeroShotClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [11]:
! ls -l {MODEL_NAME}_spark_nlp_openvino

total 1592412
-rw-r--r-- 1 root root 1630614420 Jun 18 01:48 bart_classification_openvino
drwxr-xr-x 5 root root       4096 Jun 18 01:47 fields
drwxr-xr-x 2 root root       4096 Jun 18 01:47 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny BartForZeroShotClassification model 😊

In [12]:
sequenceClassifier_loaded = BartForZeroShotClassification.load("./{}_spark_nlp_openvino".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [13]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
  .setInputCols("document") \
  .setOutputCol("token")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    sequenceClassifier_loaded
])

df = spark.createDataFrame([["I have a problem with my iPhone that needs to be resolved asap!!"],
        ["Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app."],
        ["I have a phone and I love it!"],
        ["I really want to visit Germany and I am planning to go there next year."],
        ["Let's watch some movies tonight! I am in the mood for a horror movie."],
        ["Have you watched the match yesterday? It was a great game!"],
        ["We need to hurry up and get to the airport. We are going to miss our flight!"]], ["text"])

model = pipeline.fit(df)
result = model.transform(df)

result.select("text", "class.result").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------+--------+
|text                                                                                                          |result  |
+--------------------------------------------------------------------------------------------------------------+--------+
|I have a problem with my iPhone that needs to be resolved asap!!                                              |[urgent]|
|Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app.|[music] |
|I have a phone and I love it!                                                                                 |[urgent]|
|I really want to visit Germany and I am planning to go there next year.                                       |[urgent]|
|Let's watch some movies tonight! I am in the mood for a horror movie.                                         |[music] |
|Have you watched the ma

That's it! You can now go wild and use hundreds of `BartForZeroShotClassification` models from HuggingFace 🤗 in Spark NLP 🚀
